<a href="https://colab.research.google.com/github/MalshaPiumini/FYP2022/blob/test-01/finalPreprocessingCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **I M P O R T S**

In [ ]:
!pip install unidecode
!pip install emoji --upgrade

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from unidecode import unidecode
import numpy as np
import random
import re
import psycopg2 as ps
import string
import emoji
import joblib

In [ ]:
print(emoji.emojize('Python es :pulgar_hacia_arriba:', language='es'))

# **O P E N__D A T A S E T ✌**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
hateSpeechData = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FYP Implementations/March-test2.csv" )

#Setting the column width
pd.set_option('display.max_colwidth', 100)

#Setting the column names
hateSpeechData.columns = ['Text','Feeling', 'Emotion','Label']

#Printing first 5 rows
hateSpeechData.head()

In [ ]:
# check the shape of training and testing dataset
print(hateSpeechData.shape)

In [ ]:
# Check null or NaN values are present in the training data and testing data
print(hateSpeechData.isnull().values.any())

In [ ]:
#shape of the dataset
print("Input data has {} rows and {} columns".format(len(hateSpeechData), len(hateSpeechData.columns)))

In [ ]:
# How many cb/non cb texts are there?
print("Out of {} rows, {} are Cyberbullying, {} are Not-cyberbullying".format(len(hateSpeechData),
                                                       len(hateSpeechData[hateSpeechData['Label']==1]),
                                                       len(hateSpeechData[hateSpeechData['Label']==0])))

In [ ]:
#Converting all the labels to upper case
hateSpeechData['Feeling'] = hateSpeechData['Feeling'].str.lower()
hateSpeechData['Emotion'] = hateSpeechData['Emotion'].str.lower()
hateSpeechData.head()

In [ ]:
#Feelings in the dataset
print("Out of {} rows, {} are Anger, {} are Suprise, {} are Happy, {} are Sad, {} are Disgust, {} are Neutral".format(len(hateSpeechData),
                                                       len(hateSpeechData[hateSpeechData['Feeling']=='anger']),
                                                       len(hateSpeechData[hateSpeechData['Feeling']=='suprise']),
                                                       len(hateSpeechData[hateSpeechData['Feeling']=='happy']),
                                                       len(hateSpeechData[hateSpeechData['Feeling']=='sad']),
                                                       len(hateSpeechData[hateSpeechData['Feeling']=='disgust']),
                                                       len(hateSpeechData[hateSpeechData['Feeling']=='neutral'])))

In [ ]:
#Emotions in the dataset
print("Out of {} rows, {} are Anger, {} are Suprise, {} are Happy, {} are Sad,{} are Disgust, {} are Neutral".format(len(hateSpeechData),
                                                       len(hateSpeechData[hateSpeechData['Emotion']=='anger']),
                                                       len(hateSpeechData[hateSpeechData['Emotion']=='suprise']),
                                                       len(hateSpeechData[hateSpeechData['Emotion']=='happy']),
                                                       len(hateSpeechData[hateSpeechData['Emotion']=='sad']),
                                                       len(hateSpeechData[hateSpeechData['Emotion']=='disgust']),
                                                       len(hateSpeechData[hateSpeechData['Emotion']=='neutral'])))

In [ ]:
# Reading the dataset row by row
from csv import reader
# skip first line i.e. read header first and then iterate over each row od csv as a list
with open('/content/drive/MyDrive/Colab Notebooks/FYP Implementations/March-test2.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    header = next(csv_reader)
    if header != None:
        for row in csv_reader:
            print(row)

# **C L E A N I N G ✌**

In [ ]:
import pandas as pd
import emoji as emj

EMOJIS = emj.UNICODE_EMOJI["en"]

#Move emojies to another column
def add_emoji_column(df):
  emoji_column = ""
  for emoji in EMOJIS:
    if emoji in df:
      emoji_column = emoji_column + emoji
  return emoji_column

In [ ]:
hateSpeechData['emoji']=hateSpeechData['Text'].apply(lambda x : add_emoji_column(x))
hateSpeechData.head()

In [ ]:
# Remove Emojies from the text
def remove_emojies(df):
  cleaned_emojies = ""
  for emoji in EMOJIS:
    if emoji in df:
      df = df.replace(emoji, " ")
  return df

In [ ]:
hateSpeechData['emoji_cleaned_text']=hateSpeechData['Text'].apply(lambda x : remove_emojies(x))
hateSpeechData.head() 

In [ ]:
#Sinhala Unicode -> Romanized Sinhala
def sinhalaToSinglish(inputDf):
  for index in range(len(inputDf)):
      currTokenList = inputDf['emoji_cleaned_text'].values[index]
      singlishTokenList = []
      for token in currTokenList:
        singlishTokenList.append(unidecode(token))
        inputDf['emoji_cleaned_text'].values[index] = "".join(singlishTokenList);
  return inputDf

In [ ]:
sinhalaToSinglish(hateSpeechData)
hateSpeechData.head()

In [ ]:
hateSpeechData['emoji_cleaned_text'] = hateSpeechData['emoji_cleaned_text'].str.lower()

In [ ]:
#Remove URLs
def urlRemoving(inputDf):
  for i in range(len(inputDf)):
    currentPhase= inputDf['emoji_cleaned_text'].values[i]
    inputDf['emoji_cleaned_text'].values[i] = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', currentPhase)
  return inputDf 

In [ ]:
#Training Dataset
urlRemoving(hateSpeechData)
hateSpeechData.head()

In [ ]:
# Clean training data and testing data
# remove html tags
def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result

# remove line seperators
def remove_line_seperators(string):
    result=re.sub("\\n", "", string)
    return result

# remove repeated characters
def remove_repeated_characters(text):
    regex_pattern = re.compile(r'(.)\1+')
    clean_text = regex_pattern.sub(r'\1\1', text)
    return clean_text

# # seperate digit with text
# def separate_digit_text(text):
#     regex_patter = re.compile(r'([\d]+)([a-zA-Z]+)')
#     clean_text = regex_patter.sub(r'\1 \2', text)
#     return clean_text

# remove punctuations
def remove_punctuations(text):
    test_punc_removed = [(ch if ch not in string.punctuation else " ") for ch in text ]
    test_punc_removed_join = ''.join(test_punc_removed)
    return test_punc_removed_join

# remove extra space
def remove_extra_space(text):
    clean_text = ' '.join(text.strip().split())
    return clean_text

# clean all text
def cleantext(text):
    text_cleaned= remove_line_seperators(
            remove_extra_space(
                remove_punctuations(
                    remove_repeated_characters(remove_tags(text)            
                            )
                        )
                    )
                )
    # text_cleaned=separate_digit_text(text_cleaned)
    return text_cleaned

In [ ]:
#clean training data
hateSpeechData['comments_fixed']=hateSpeechData['emoji_cleaned_text'].apply(lambda cw : cleantext(cw))
hateSpeechData.head()

# **T O K E N I Z E**

In [ ]:
def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

In [ ]:
#Tokenize training data
trainingData['body_text_tokenized'] = trainingData['comments_fixed'].apply(lambda x: tokenize(x.lower()))
trainingData.head()

In [ ]:
#Tokenize testing data
testingData['body_text_tokenized'] = testingData['comments_fixed'].apply(lambda x: tokenize(x.lower()))
testingData.head()

# **S T O P__W O R D__R E M O V A L**

In [ ]:
# function to count the total tokens
def getTotalTokens(token_set):
    count = 0
    for tSet in token_set:
        count += len(tSet)
    return count

In [ ]:
# function to count the total unique tokens
def getTotalUniqueTokens(token_set):
    count = 0
    for tSet in token_set:
        uniqueTokens = set(tSet)
        count += len(uniqueTokens)
    return count

In [ ]:
#-------------------------COMPARE REMOVING AND WITHOUT REMOVING----------------------------
#Remove Stop words
stopword = ['dha', 'ya', 'mea', 'ea', 'haa', 'ma', 'bawa', 'nam', 'dhe', 'shdhaha', 'yi',
                                'shaha', 'lesa', 'hoo', 'aetha', 'nisa', 'meama', 'wa', 'gaena', 'wisin', 'wita',
                                'ema', 'thula', 'kota', 'yuthu', 'washayen', 'no', 'pilibadha', 'anuwa', 'eheth',
                                'pasu', 'eak', 'mean', 'sita', 'neatha', 'weani', 'nea', 'wada', 'bohoa', 'etha',
                                'naehea', 'thawath', 'bawata', 'pamanak', 'yam', 'samaga', 'matha', 'sae', 'namuth',
                                'peara', 'maha', 'weatha', 'pamana', 'samaga', 'nowea', 'magin', 'enam', 'weanuwen',
                                'yatathea', 'saema', 'pasuwa', 'thamayi', 'dhakwa', 'nowa', 'siyalu', 'yaeyi',
                                'aethaem', 'yanu', 'waenuwen', 'haema', 'pinisa', 'patan', 'sahitha', 'hari',
                                'aethulu', 'wagae', 'paridhi', 'pawa', 'thawa', 'samaharu', 'samahara', 'eath',
                                'wena', 'baewin', 'pahala', 'wadaath', 'haera', 'pita', 'eweni', 'aa', 'ehala',
                                'thulin', 'ekka', 'aadhi', 'mulu', 'sambandhayean', 'peawathi', 'oya', 'wath',
                                'pamani', 'wen', 'pura', 'ona', 'epa', 'ewita', 'hewath', 'yanuwen', 'meawaeni',
                                'ow', 'ethi', 'tika', 'ihatha', 'kara', 'kisiyam', 'aeyi', 'kisima', 'ara', 'heyin',
                                'kisidhu', 'hata', 'anek', 'idhiri', 'thek', 'waenuwata', 'haetiyata', 'yata',
                                'dhaeyi', 'mokadha', 'maghin', 'baew', 'an', 'ona', 'laga', 'uth', 'haraha',
                                'asala', 'baehaera', 'hoth', 'ekthara', 'uda', 'athurin', 'klhi', 'ayuru', 'noyek',
                                'atharin', 'ammea', 'aethuluwa', 'hariyatama', 'ethana', 'ehaata', 'kawara',
                                'wanaahi', 'osseama', 'naethinam', 'udhesa', 'naedha', 'baehae', 'misa', 'haeti',
                                'wata', 'bae', 'etharam', 'thawadhuratath', 'thawa', 'thuru', 'idhiriyae',
                                'edhiriyae', 'hariyata', 'ebadhu', 'naethdha', 'naedhdha', 'anea', 'mona',
                                'ethamath', 'ithamath', 'ethakota', 'aneakuth', 'awata', 'aetha', 'meabadhu',
                                'lath', 'thulata', 'kohomadha', 'kumana', 'lesin', 'badhu', 'oonaema', 'sahagatha',
                                'lagata', 'wethata', 'waenidhaa', 'onea', 'passea', 'thissea', 'saeti', 'baegin',
                                'sesu',
                                'bohoma', 'kotharam', 'aethath', 'yanuwen', 'koyi', 'upa', 'yae', 'kotharam', 'o',
                                'anith', 'onna', 'naethahoth', 'witharak', 'kawdha', 'hugak', 'naetheyi', 'pitatha',
                                'saema', 'nisama', 'passea', 'seaka', 'ewan', 'monawadha', 'samagama', 'thawadha',
                                'sadha', 'haebaeyi', 'atharin', 'lesata', 'dhigata', 'mandha', 'namuth', 'namudhu',
                                'eheanam', 'kepa', 'emagin', 'idhin', 'mokakdha', 'dhigea', 'witaka', 'meani',
                                'sambandhawa', 'nohoth', 'anna', 'monawadha', 'harima', 'naethinam', 'atharaata',
                                'pitathata', 'wan', 'yatin', 'wagea', 'nisath', 'pahalata', 'anya', 'eha', 'witama',
                                'vitama', 'noyekuth', 'withara',
                                'ihalin', 'iwatha', 'nomethiwa', 'hudeak', 'meaweni', 'waethin', 'weni', 'waewa',
                                'andhamin', 'meanma', 'was', 'kiipa', 'kochchra', 'methaenin', 'methena',
                                'idhiripita', 'purama', 'witaka', 'athishaya', 'dhepasa', 'peareadha', 'aethulata',
                                'atharea', 'yamkisi', 'aassithawa', 'thak', 'koa', 'uda', 'meatha', 'ehalata',
                                'dhenatamath', 'wenama', 'arabeaya', 'rahitha', 'pitin', 'paahea', 'aethinam',
                                'aeththam', 'atharam', 'kuwuruth', 'paridhdhen', 'watea', 'keabadhu', 'kothek',
                                'naththam', 'hindha', 'pasupasa', 'meha', 'ladheyi', 'paasa', 'pawa', 'perata',
                                'waediputa', 'aethin', 'ethek', 'vina', 'maedhin', 'idhiriyean', 'pitupasa',
                                'samaga', 'mae', 'la', 'maenawi', 'sampanna', 'athurean', 'meathram',
                                'obbata', 'nomilea', 'arabaya', 'abiyasa', 'idhiripasa', 'enamuth',
                                'mathin''issaraha', 'echchrara', 'ekko', 'behi', 'magin', 'magini', 'ketuwa',
                                'misak', 'eheamyi', 'hindha', 'idhin', 'lagin', 'asalin', 'dho''kelinma',
                                'virahitha', 'mathata', 'asalata', 'atharahura',
                                'mealagata', 'kethek', 'kola', 'ayyo', 'nomilea', 'ethanata', 'mulata', 'lu',
                                'samagi', 'che', 'nithi', 'wenidha', 'apoyi', 'ochchrata', 'aaubowan', 'huga',
                                'wanathuru', 'pamanata', 'pahathata', 'ewaka', 'pasakin', 'aethulea', 'yatathata',
                                'yabadha', 'nan',
                                'shik', 'shek', 'usi', 'anan', 'manan', 'ananmanan', 'ahaa', 'adhdharata',
                                'monayam', 'athana', 'aethule', 'aethulen', 'pahathin', 'virahithawa', 'aayi',
                                'enayin', 'samagin', 'obbehi', 'othana', 'ammo', 'sha', 'neathahoth', 'atharamaga',
                                'asabada', 'yabadhawa', 'seakwa', 'atharamaga', 'obben', 'adhdhara',
                                'achchrara', 'dhoho', 'chi', 'chik', 'adho', 'soshae', 'ethira', 'koyibata', 'ohea',
                                'oh', 'adhomeyi', 'uhu', 'uu', 'u', 'otharam', 'abiyasin', 'appachchiyea', 'ado',
                                'aane', 'ane', 'hindhama', 'ha', 'hapura', 'latha', 'ohe', 'an', 'ankisi', 'hurea',
                                'uuyi', 'adhdharin', 'embala', 'abiyasa', 'abiyasata',
                                'uuh', 'shi', 'sisi', 'koyibata', 'satapata', 'abiyesa', 'heleyiya', 'sata',
                                'satata'

                                ]

def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]
    #text =" ".join( [word for word in tokenized_list if word not in stopword])
    return text

In [ ]:
#Stop word removal testing data
hateSpeechData['body_text_nostop'] = hateSpeechData['body_text_tokenized'].apply(lambda x: remove_stopwords(x))
hateSpeechData.head(100)

# **F E A T U R E__E X T R A C T I O N**

# **V E C T O R I Z I N G**